In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
import re

In [ ]:
folder_path = '/content/drive/MyDrive/MCR_UNET/Dataset'


def calculate_cloud_cover(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Tidak ada gambar {image_path}")
        return 0

    height, width, _ = img.shape
    total_pixels = height * width
    white_pixels = np.sum(img > 159)
    cloud_cover_percentage = (white_pixels / total_pixels) * 100
    return cloud_cover_percentage

for root, dirs, files in os.walk(folder_path):
    for file in files:
        if re.search(r'_mask\.tif$', file):
            continue

        file_path = os.path.join(root, file)
        cloud_cover = calculate_cloud_cover(file_path)

        if cloud_cover > 60:
            print(f"Menghapus {file_path} dengan cloud cover {cloud_cover:.2f}% ⛔")
            os.remove(file_path)

            # Hapus file mask yang sesuai
            mask_file = re.sub(r'\.tif$', '_mask.tif', file)
            mask_file_path = os.path.join(root, mask_file)
            if os.path.exists(mask_file_path):
                print(f"Menghapus file mask: {mask_file_path} 🗑️")
                os.remove(mask_file_path)
        else:
            print(f"{file_path} memiliki cloud cover {cloud_cover:.2f}% (dibawah 60%) ✅")


In [ ]:
import shutil

output_folder = '/content/drive/MyDrive/MCR_UNET/Input_Unet'

# Membuat folder untuk setiap kategori
categories = {
    '<20%': os.path.join(output_folder, '<20%'),
    '20-40%': os.path.join(output_folder, '20-40%'),
    '40-60%': os.path.join(output_folder, '40-60%'),
    '>60%': os.path.join(output_folder, '>60%')
}

for category in categories.values():
    os.makedirs(category, exist_ok=True)

def calculate_cloud_cover(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Tidak ada gambar {image_path}")
        return 0

    height, width, _ = img.shape
    total_pixels = height * width
    white_pixels = np.sum(img > 159)
    cloud_cover_percentage = (white_pixels / total_pixels) * 100
    return cloud_cover_percentage

for root, dirs, files in os.walk(folder_path):
    for file in files:
        if re.search(r'_mask\.tif$', file):
            continue

        file_path = os.path.join(root, file)
        cloud_cover = calculate_cloud_cover(file_path)

        if cloud_cover < 20:
            category_folder = categories['<20%']
        elif 20 <= cloud_cover < 40:
            category_folder = categories['20-40%']
        elif 40 <= cloud_cover < 60:
            category_folder = categories['40-60%']
        else:
            category_folder = categories['>60%']

        # Copy file ke folder kategori
        shutil.copy(file_path, os.path.join(category_folder, file))
        mask_file = re.sub(r'\.tif$', '_mask.tif', file)
        mask_file_path = os.path.join(root, mask_file)
        if os.path.exists(mask_file_path):
            shutil.copy(mask_file_path, os.path.join(category_folder, mask_file))

        print(f"{file_path} (cloud cover {cloud_cover:.2f}%) dipindahkan ke {category_folder}")


/content/drive/MyDrive/MCR_UNET/Dataset/33/0033_2021-06-17.tif (cloud cover 0.00%) dipindahkan ke /content/drive/MyDrive/MCR_UNET/Input_Unet/<20%
/content/drive/MyDrive/MCR_UNET/Dataset/33/0033_2021-06-27.tif (cloud cover 0.00%) dipindahkan ke /content/drive/MyDrive/MCR_UNET/Input_Unet/<20%
/content/drive/MyDrive/MCR_UNET/Dataset/33/0033_2021-07-12.tif (cloud cover 38.57%) dipindahkan ke /content/drive/MyDrive/MCR_UNET/Input_Unet/20-40%
/content/drive/MyDrive/MCR_UNET/Dataset/33/0033_2021-07-07.tif (cloud cover 0.00%) dipindahkan ke /content/drive/MyDrive/MCR_UNET/Input_Unet/<20%
/content/drive/MyDrive/MCR_UNET/Dataset/33/0033_2021-07-17.tif (cloud cover 16.07%) dipindahkan ke /content/drive/MyDrive/MCR_UNET/Input_Unet/<20%
/content/drive/MyDrive/MCR_UNET/Dataset/33/0033_2021-07-27.tif (cloud cover 16.61%) dipindahkan ke /content/drive/MyDrive/MCR_UNET/Input_Unet/<20%
/content/drive/MyDrive/MCR_UNET/Dataset/33/0033_2021-07-22.tif (cloud cover 0.00%) dipindahkan ke /content/drive/MyDriv

In [ ]:
import matplotlib.pyplot as plt

def display_images_from_categories(categories):
    plt.figure(figsize=(15, 21))
    for category_index, (category, path) in enumerate(categories.items()):
        images = [f for f in os.listdir(path) if f.endswith('.tif') and not f.endswith('_mask.tif')]
        selected_images = images[:7]

        for i, image_file in enumerate(selected_images):
            img_path = os.path.join(path, image_file)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            plt.subplot(7, 3, i * 3 + category_index + 1)
            plt.imshow(img)
            plt.axis('off')
            plt.title(f"{category}: {image_file}")

    plt.suptitle("Sampel Gambar dari Masing-Masing Kategori", fontsize=30)
    plt.tight_layout()
    plt.show()

display_images_from_categories(categories)


Output hidden; open in https://colab.research.google.com to view.